In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

from models.segvol.base import SegVolConfig
from models.segvol.lora_model import SegVolLoRA
from utils.dataset import VolumetricDataset

device = "cpu"

img_path = "../data/segthor_train/train/Patient_05/Patient_05.nii.gz"
gt_path = "../data/segthor_train/train/Patient_05/GT.nii.gz"

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
2024-10-07 13:14:27.237059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
config = SegVolConfig(test_mode=False)

# NOTE: The default spatial size is 32x256x256 which is not that much to capture the whole volume

model = SegVolLoRA(config).to(device)

monai.networks.nets.vit ViT.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884


In [3]:
from torch.utils.data import DataLoader

train_dataset = VolumetricDataset("../data/segthor_train/train", 0.9, model.processor, num_classes=5, cache_size=2, train=False)

# Must be batch size 1
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
print(len(train_loader))

4


In [9]:
organs_cls = ["esophagus", "heart", "trachea", "aorta"]

loss = 0

for i, obj in enumerate(train_loader):
    print(i)
    image = obj["image"].to(device)
    gt3D = obj["label"].to(device)
    
    for j in range(len(organs_cls)):
        cat = organs_cls[j]
        labels_cls = gt3D[:, j]
        loss = model.forward_train(image, train_organs=cat, train_labels=labels_cls, modality="CT") 
    
    # Good to have manual memory management here
    del obj
    break



In [11]:
%%prun
for i, obj in enumerate(train_loader):
    print(i)
    image = obj["image"].to(device)
    gt3D = obj["label"].to(device)

    del obj, image, gt3D

0
1
2
3
 

         249337 function calls (248173 primitive calls) in 34.648 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  208/128   10.786    0.052   10.796    0.084 {method 'to' of 'torch._C.TensorBase' objects}
    40/24    9.307    0.233    9.310    0.388 {built-in method torch.stack}
     9240    3.020    0.000    3.020    0.000 {method 'decompress' of 'zlib.Decompress' objects}
        4    2.160    0.540    2.160    0.540 {built-in method scipy.sparse._sparsetools.csr_todense}
        4    1.590    0.398    1.590    0.398 {method 'min' of 'torch._C.TensorBase' objects}
       28    1.410    0.050    6.384    0.228 format.py:738(read_array)
       16    1.269    0.079    1.269    0.079 {method 'max' of 'torch._C.TensorBase' objects}
        4    0.891    0.223    0.891    0.223 {built-in method numpy.fromfile}
       16    0.889    0.056    0.889    0.056 {method 'any' of 'torch._C.TensorBase' objects}
     9288    0.668    

In [ ]:
for obj in train_loader:
    image, gt3D = obj["image"].to(device), obj["label"].to(device)
    for j in range(len(organs_cls)):
        cat = organs_cls[j]
        labels_cls = gt3D[:, j]
        loss = model.forward_train(image, train_organs=cat, train_labels=labels_cls, modality="CT")